# 练习3：流水车间调度器
-------
>本节练习节选自书籍《500 lines or less》——A Flow Shop Scheduler

## 介绍

在本练习中，我们将介绍流水车间调度问题。流水车间调度问题是查找最优解问题的一种，本练习基于局部搜索（local search）方法，实现流水车间调度器。

## 1 流水车间调度
流水车间调度问题是一种优化问题，其中我们必须确定作业中各种任务的处理时间，以便调度任务以最大程度地减少完成作业所需的总时间。

例如，一家汽车制造商拥有一条装配线，其中汽车的每个零件都在不同的机器上依次完成。不同的订单可能有自定义要求，例如，使车身涂漆的任务因一辆汽车而异。

在我们的示例中，每辆汽车都是新**作业**，每辆汽车的零件都称为**任务**。每个作业将具有相同的任务序列来完成。

流水车间调度的目的是最大程度地减少处理从每个作业到完成所有任务的总时间。（通常，此总时间称为完成时间。）此问题有很多应用程序，但与优化生产设备最相关。

每个流水车间都由$n$台机器和$m$个任务组成。在我们的示例中，将有$n$个汽车工作站，总共有$m$辆汽车。每个作业都恰好由$n$个任务组成。我们可以假设一个作业的第$i$个任务必须使用机器$i$，并且需要预定数量的处理时间：$p(i,j)$是第$j$个作业的第$i$个任务的处理时间。此外，给定作业的任务顺序必须遵循可用机器的顺序。对于给定的作业，任务$i$必须在任务$i+1$开始之前完成。在我们的示例中，我们不想在组装框架之前就开始为汽车刷漆。最终的限制就是**不允许在一台机器上同时处理两个任务**。

因为作业的任务顺序是预先确定的，所以可以将流水车间调度问题的解决方法看作是作业的排列。对于每台机器，在一台机器上处理的作业顺序将是相同的，并且经过排列后，作业$j$中的机器$i$的任务计划为以下两种情况：

- 作业$j-1$中机器$i$的任务完成（即同一机器上的最新任务）
- 作业$j$中机器$i-1$的任务完成（即同一任务上的最新任务）

因为我们选择这两个值中的最大值，所以将创建机器i或作业j的空闲时间。我们最终希望最小化此空闲时间，因为它将使总有效期变大。

由于问题的简单形式，作业的任何排列都是有效的解决方案，而最佳解决方案将对应于某些排列。因此，我们通过更改作业的排列并测量相应的有效期来寻求改进的解决方案。在下文中，我们将作业的排列称为候选。

让我们考虑一个简单的示例，其中包含两个作业和两个计算机。第一项任务有任务A和B，分别需要1分钟和2分钟才能完成。第二项任务有任务C和D，分别需要2分钟和1分钟才能完成。回想一下，A必须先于B而C必须先于D。因为有两个工作，所以我们只考虑两个排列。如果在工作1之前订购工作2，则制造跨度为5（图1）；

![](3-1.png)

另一方面，如果我们在作业2之前订购作业1，则制造跨度仅为4（图2）。
![](3-2.png)

请注意，没有多余的空间可以提前完成任何任务。良好排列的指导原则是最大程度地减少任何机器无需处理的时间。

### 1.1 局部搜索

当最佳解决方案难以计算时，局部搜索是解决优化问题的策略。直观地讲，它从一个看起来不错的解决方案转变为另一个看起来更好的解决方案。我们没有考虑所有可能的解决方案作为下一个关注点，而是定义了所谓的社区：被认为与当前解决方案相似的解决方案集。因为作业的任何排列都是有效的解决方案，所以我们可以将任何将作业混洗的机制视为本地搜索过程（实际上，这是我们在下面所做的事情）。

要正式使用本地搜索，我们必须回答以下几个问题：

- 我们应该从什么解决方案开始？
- 给定一个解决方案，我们应该考虑哪些邻近解决方案？
- 给定候选邻居的集合，我们应该考虑移到下一个？

以下三个部分依次解决了这些问题。

## 2 通用求解器
在本节中，我们提供了流水车间调度程序的一般框架。首先，我们需要导入所需的python库文件和求解器的相关参数设置：

In [1]:
import sys, os, time, random

from functools import partial
from collections import namedtuple
from itertools import product

import neighbourhood as neigh
import heuristics as heur

##############
## Settings ##
##############

TIME_LIMIT = 300.0 # Time (in seconds) to run the solver
TIME_INCREMENT = 13.0 # Time (in seconds) in between heuristic measurements
DEBUG_SWITCH = False # Displays intermediate heuristic info when True
MAX_LNS_NEIGHBOURHOODS = 1000 # Maximum number of neighbours to explore in LNS

其中，`TIME_INCREMENT`是用作动态策略选择的一部分，`MAX_LNS_NEIGHBOURHOODS`是用作邻域选择的最大数量限制。接下来会有更加详细的描述。

这些参数的设置也可以作为命令行参数显示给用户，此处我们改为将输入数据作为提供给程序。输入的问题（来自Taillard基准集）被假定为流水车间调度的标准格式。

以下代码为`__main__`求解器文件的方法，根据输入到程序的参数数量来选择调用合适的函数：

```python
if __name__ == '__main__':

    if len(sys.argv) == 2:
        data = parse_problem(sys.argv[1], 0)
    elif len(sys.argv) == 3:
        data = parse_problem(sys.argv[1], int(sys.argv[2]))
    else:
        print("\nUsage: python flow.py <Taillard problem file> [<instance number>]\n")
        sys.exit(0)

    (perm, ms) = solve(data)
    print_solution(data, perm)
```
在上述代码中：
- `parse_problem`函数为Taillard基准集问题文件的解析函数
- `solve`函数为求解函数
- `print_problem`函数为最终结果的输出函数

### 2.1 求解逻辑
我们先来看`solve`函数，该方法期望`data`变量是一个整数列表，其中包含每个作业的活动持续时间。其开始于初始化一组全局策略。关键是我们使用`start_*`来命名的变量，这些变量用来维护每个策略的统计数据，有助于在求解过程中动态选择策略。

其中：
- `start_improvements`: 通过某策略可以改善解决方案的数量
- `start_time_spent`: 在某策略上花费的时间
- `start_weights`: 与某策略的好坏相对应的权重
- `start_usage`: 我们使用某策略的次数

```python
def solve(data):
    """Solves an instance of the flow shop scheduling problem"""

    # We initialize the strategies here to avoid cyclic import issues
    initialize_strategies()
    global STRATEGIES

    strat_improvements = {strategy: 0 for strategy in STRATEGIES}
    strat_time_spent = {strategy: 0 for strategy in STRATEGIES}
    strat_weights = {strategy: 1 for strategy in STRATEGIES}
    strat_usage = {strategy: 0 for strategy in STRATEGIES}
```

流水车间调度问题的一个特征是：每个排列都是有效的解决方案，并且至少有一个将具有最佳的制造期。值得庆幸的是，当从一种排列转换到另一种排列时，这可以使我们放弃检查是否处于可行解的空间之内，因为所有一切都是可行的。

但是，要在置换空间中开始局部搜索时，我们必须**具有初始置换**。为简单起见，我们选择**随机排列作业列表**来初始化局部搜索。

```python
    perm = range(len(data))
    random.shuffle(perm)
```

接下来，我们需要对变量进行初始化，以使我们能够跟踪到目前为止找到的最佳排列以及提供输出的时序信息。

```python
    # Keep track of the best solution
    best_make = makespan(data, perm)
    best_perm = perm
    res = best_make

    # Maintain statistics and timing for the iterations
    iteration = 0
    time_limit = time.time() + TIME_LIMIT
    time_last_switch = time.time()

    time_delta = TIME_LIMIT / 10
    checkpoint = time.time() + time_delta
    percent_complete = 10

    print("\nSolving...")
```

由于这是局部搜索解决方案，因此只要没有达到时间限制，我们就可以继续尝试并改进解决方案。我们提供指示求解器进度的输出，并跟踪我们计算出的迭代次数：
```python
    while time.time() < time_limit:

        if time.time() > checkpoint:
            print " %d %%" % percent_complete
            percent_complete += 10
            checkpoint += time_delta

        iteration += 1
```

下面我们描述如何选择策略，但是到目前为止，知道策略提供了两个功能库：
- 一个`neighbourhood`功能为我们提供了一组下一个要考虑的候选人
- 一个`heuristic`功能则从集合中选择了最佳候选人

从这些函数中，我们得到一个新的排列（`perm`）和一个新的制造期结果（`res`）：
```python
        # Heuristically choose the best strategy
        strategy = pick_strategy(STRATEGIES, strat_weights)

        old_val = res
        old_time = time.time()

        # Use the current strategy's heuristic to pick the next permutation from
        # the set of candidates generated by the strategy's neighbourhood
        candidates = strategy.neighbourhood(data, perm)
        perm = strategy.heuristic(data, candidates)
        res = makespan(data, perm)
```

计算工期的代码非常简单：我们可以通过评估最终作业的完成时间来从排列中进行计算。

In [5]:
def makespan(data, perm):
    """Computes the makespan of the provided solution"""
    return compile_solution(data, perm)[-1][-1] + data[perm[-1]][-1]

我们将在后面看到`compile_solution`工作原理，但是现在足以知道已返回2D数组，并且坐标为[-1][-1]的元素与计划中最终作业的开始时间相对应。

为了帮助选择策略，我们保留以下统计数据：
1. 该策略改善了解决方案的数量；
2. 该策略花费了多少时间计算信息；
3. 使用该策略的次数。如果偶然发现更好的解决方案，我们还将更新变量以获得最佳排列：

```python
        # Record the statistics on how the strategy did
        strat_improvements[strategy] += res - old_val
        strat_time_spent[strategy] += time.time() - old_time
        strat_usage[strategy] += 1

        if res < best_make:
            best_make = res
            best_perm = perm[:]
            
```
定期更新用于策略的统计信息。为了便于阅读，我们删除了相关代码段，并在下面详细说明了代码。

作为最后一步，一旦`while`循环完成（即达到了时间限制），我们将输出有关求解过程的一些统计信息，并返回最佳排列及其`makepan`：
```python
    print(" %d %%\n" % percent_complete)
    print("\nWent through %d iterations." % iteration)

    print("\n(usage) Strategy:")
    results = sorted([(strat_weights[STRATEGIES[i]], i)
                      for i in range(len(STRATEGIES))], reverse=True)
    for (w, i) in results:
        print("(%d) \t%s" % (strat_usage[STRATEGIES[i]], STRATEGIES[i].name))

    return (best_perm, best_make)
```

最终的求解函数`solve`为：

In [6]:
def solve(data):
    """Solves an instance of the flow shop scheduling problem"""

    #我们在这里初始化策略以避免周期性导入问题
    initialize_strategies()
    global STRATEGIES
    
    strat_improvements = {strategy: 0 for strategy in STRATEGIES}
    strat_time_spent = {strategy: 0 for strategy in STRATEGIES}
    strat_weights = {strategy: 1 for strategy in STRATEGIES}
    strat_usage = {strategy: 0 for strategy in STRATEGIES}

    #随机开始
    perm = list(range(len(list(data))))
    random.shuffle(perm)

    #记录最优策略
    best_make = makespan(data, perm)
    best_perm = perm
    res = best_make

    #维护迭代的统计信息和时序
    iteration = 0
    time_limit = time.time() + TIME_LIMIT
    time_last_switch = time.time()

    time_delta = TIME_LIMIT / 10
    checkpoint = time.time() + time_delta
    percent_complete = 10

    print ("\nSolving...")

    while time.time() < time_limit:

        if time.time() > checkpoint:
            print (" %d %%" % percent_complete)
            percent_complete += 10
            checkpoint += time_delta

        iteration += 1

        # 启发式的选择最优策略
        strategy = pick_strategy(STRATEGIES, strat_weights)

        old_val = res
        old_time = time.time()

        # 使用当前策略的heuristic方法，从策略邻域生成的候选集合中选择下一个排列
        candidates = strategy.neighbourhood(data, perm)
        perm = strategy.heuristic(data, candidates)
        res = makespan(data, perm)

        #记录有关策略执行情况的统计信息
        strat_improvements[strategy] += res - old_val
        strat_time_spent[strategy] += time.time() - old_time
        strat_usage[strategy] += 1

        if res < best_make:
            best_make = res
            best_perm = perm[:]

        #定期更改可用策略的权重，这样搜索可以动态地转向最近被证明更有效的策略。
        if time.time() > time_last_switch + TIME_INCREMENT:

            #将改进所需的时间归一化
            results = sorted([(float(strat_improvements[s]) / max(0.001, strat_time_spent[s]), s)
                              for s in STRATEGIES])

            if DEBUG_SWITCH:
                print ("\nComputing another switch...")
                print ("Best performer: %s (%d)" % (results[0][1].name, results[0][0]))
                print ("Worst performer: %s (%d)" % (results[-1][1].name, results[-1][0]))

            #提升成功策略的分量
            for i in range(len(STRATEGIES)):
                strat_weights[results[i][1]] += len(STRATEGIES) - i

                #此外，加强未使用的策略以避免饥饿
                if results[i][0] == 0:
                    strat_weights[results[i][1]] += len(STRATEGIES)

            time_last_switch = time.time()

            if DEBUG_SWITCH:
                print (results)
                print (sorted([strat_weights[STRATEGIES[i]] for i in range(len(STRATEGIES))]))

            strat_improvements = {strategy: 0 for strategy in STRATEGIES}
            strat_time_spent = {strategy: 0 for strategy in STRATEGIES}


    print (" %d %%\n" % percent_complete)
    print ("\nWent through %d iterations." % iteration)

    print ("\n(usage) Strategy:")
    results = sorted([(strat_weights[STRATEGIES[i]], i)
                      for i in range(len(STRATEGIES))], reverse=True)
    for (w, i) in results:
        print ("(%d) \t%s" % (strat_usage[STRATEGIES[i]], STRATEGIES[i].name))

    return (best_perm, best_make)

### 2.2 问题解析

作为解析过程的输入，我们提供了可在其中找到输入的文件名以及应使用的示例编号。（每个文件包含许多实例，如下图所示。）
<img src="3-3.png" style="weigh:270px,heigh:270px">

我们通过读取文件并标识分隔每个问题实例的行来开始解析：
```python
def parse_problem(filename, k=1):
    with open(filename, 'r') as f:
        # Identify the string that separates instances
        problem_line = ('/number of jobs, number of machines, initial seed, '
                        'upper bound and lower bound :/')

        # Strip spaces and newline characters from every line
        lines = map(str.strip, f.readlines())    
```
为了更容易查找正确的实例，我们假设行将以'/'字符分隔。这样，我们就可以根据出现在每个实例顶部的通用字符串来分割文件，并且在第一行的开头添加一个'/'字符可以使得无论我们选择哪种实例，下面的字符串处理工作都会正常进行。

给定文件中找到的实例集合，我们还将检测提供的实例号何时超出范围。
```python
        # We prep the first line for later
        lines[0] = '/' + lines[0]

        # We also know '/' does not appear in the files, so we can use it as
        #  a separator to find the right lines for the kth problem instance
        try:
            lines = '/'.join(lines).split(problem_line)[k].split('/')[2:]
        except IndexError:
            max_instances = len('/'.join(lines).split(problem_line)) - 1
            print("\nError: Instance must be within 1 and %d\n" % max_instances)
            sys.exit(0)
```

我们直接解析数据，将每个任务的处理时间转换为整数并将其存储在列表中。最后，我们压缩数据以反转行和列，以使格式符合上述求解代码的期望。（其中的每个项目data都应对应于特定的工作。）
```python
        # Split every line based on spaces and convert each item to an int
        data = [map(int, line.split()) for line in lines]

    # We return the zipped data to rotate the rows and columns, making each
    #  item in data the durations of tasks for a particular job
    return zip(*data)
```

最终的解析函数`parse_problem`为：

In [7]:
def parse_problem(filename, k=1):
    """
    解析Taillard问题文件的第k个实例
    Taillard问题文件是流水车间调度问题的标准基准集。
    """

    print ("\nParsing...")

    with open(filename, 'r') as f:
        #标识分隔实例的字符串
        problem_line = '/number of jobs, number of machines, initial seed, upper bound and lower bound :/'

        #删除每行中的空格和换行符
        lines = list(map(str.strip, f.readlines()))

        # We prep the first line for later
        lines[0] = '/' + lines[0]

        #我们也知道'/'不会出现在文件中
        #因此我们可以将其用作分隔符以查找第k个问题实例的正确行
        try:
            lines = '/'.join(lines).split(problem_line)[k].split('/')[2:]
        except IndexError:
            max_instances = len('/'.join(lines).split(problem_line)) - 1
            print ("\nError: Instance must be within 1 and %d\n" % max_instances)
            sys.exit(0)

        #根据空格分割每一行并将每一项转换为一个整数
        data = [map(int, line.split()) for line in lines]

    #我们返回压缩后的数据以旋转行和列
    #从而使数据中的每个项目都成为特定任务的任务持续时间
    return zip(*data)

### 2.3 编译解决方案
流水车间调度问题的解决方案包括为每个作业中的每个任务提供精确的时间安排。因为我们隐含地表示工作的排列，所以我们引入了`compile_solution`将排列转换为精确时间的函数。

作为输入，该函数接收问题数据（为我们提供每个任务的持续时间）和工作排列的数据。
该函数首先初始化用于存储每个任务的开始时间的数据结构，然后将第一个作业中的任务包括在排列中。

然后，我们为其余作业添加所有任务。

作业中的第一个任务将始终在上一个作业中的第一个任务完成后立即开始。对于其余任务，其开始时间是**同一作业中上一个任务的完成时间和同一台计算机上上一个任务的完成时间中的最大值**。
最终返回每个任务的时间安排。

该函数代码如下：

In [8]:
def compile_solution(data, perm):
    """编译给定作业排列的机器上的调度"""
    num_machines = len(list(data[0]))

    #注意，使用[[]]* m是不正确的
    #因为它只是复制相同的列表m次(而不是创建m个不同的列表)。
    machine_times = [[] for _ in range(num_machines)]

    #将初始作业分配给机器
    machine_times[0].append(0)
    for mach in range(1,num_machines):
        #前一个任务完成后，开始工作中的下一个任务
        machine_times[mach].append(machine_times[mach-1][0] +
                                   data[perm[0]][mach-1])

    #分配剩余的工作
    for i in range(1, len(perm)):

        #第一台机器从不包含任何空闲时间
        job = perm[i]
        machine_times[0].append(machine_times[0][-1] + data[perm[i-1]][0])

        #对于其余的机器，启动时间是作业中的前一个任务何时完成，
        #或当前机器何时完成前一个作业的任务的最大值。
        for mach in range(1, num_machines):
            machine_times[mach].append(max(machine_times[mach-1][i] + data[perm[i]][mach-1],
                                        machine_times[mach][i-1] + data[perm[i-1]][mach]))

    return machine_times

### 2.4 印刷解决方案

求解过程完成后，程序将以紧凑形式输出有关解决方案的信息。我们没有提供每项任务的每个任务的准确时间安排，而是输出以下信息：

1. 产生最佳生产期的工作排列
2. 排列的计算的计算跨度
3. 每台机器的开始时间，完成时间和空闲时间
4. 每个作业的开始时间，完成时间和空闲时间

其中：

- **作业或机器的开始时间**对应于作业或机器上第一个任务的开始。
- **作业或机器的完成时间**对应于作业或机器上最终任务的结束。
- **空闲时间**是特定作业或机器的任务之间的松弛时间。

理想情况下，我们希望减少空闲时间，因为这意味着总的处理时间也将减少。

我们已经讨论了用于编译解决方案的代码（即，计算每个任务的开始时间），并且输出排列和`makepan`。

接着，我们使用Python中的字符串格式化功能来打印每个机器和作业的开始，结束和空闲时间表。

请注意，作业的空闲时间是从作业开始到完成为止的时间，减去该作业中每个任务的处理时间之和。我们以类似的方式计算机器的空闲时间。

In [9]:
def print_solution(data, perm):
    """打印计算出的解决方案的统计信息"""

    sol = compile_solution(data, perm)

    print ("\nPermutation: %s\n" % str([i+1 for i in perm]))

    print ("Makespan: %d\n" % makespan(data, perm))

    row_format ="{:>15}" * 4
    print( row_format.format('Machine', 'Start Time', 'Finish Time', 'Idle Time'))
    for mach in range(len(data[0])):
        finish_time = sol[mach][-1] + data[perm[-1]][mach]
        idle_time = (finish_time - sol[mach][0]) - sum([job[mach] for job in data])
        print (row_format.format(mach+1, sol[mach][0], finish_time, idle_time))

    results = []
    for i in range(len(data)):
        finish_time = sol[-1][i] + data[perm[i]][-1]
        idle_time = (finish_time - sol[0][i]) - sum([time for time in data[perm[i]]])
        results.append((perm[i]+1, sol[0][i], finish_time, idle_time))

    print ("\n")
    print( row_format.format('Job', 'Start Time', 'Finish Time', 'Idle Time'))
    for r in sorted(results):
        print (row_format.format(*r))

    print( "\n\nNote: Idle time does not include initial or final wait time.\n")

## 3 策略
接下来，我们对本练习开始时所使用的
```
import neighbourhood as neigh
import heuristics as heur
```
中的两个功能类进行介绍，并使用代码实现如何进行动态策略的选择。

### 3.1 Neighbourhood
局部搜索的思想是将局部解决方案从一个解决方案迁移到附近的其他解决方案。我们将给定解决方案的**邻域**称为局部的其他解决方案。

在本节中，我们详细介绍四个潜在的领域，每个领域的复杂性都在不断提高。

**第一邻域产生给定数量的随机排列。**这个邻里甚至没有考虑我们开始时使用的解决方案，因此“邻里”一词扩展了事实。但是，在搜索中包括一些随机性是一种好习惯，因为它可以促进对搜索空间的探索。

In [10]:
def neighbours_random(data, perm, num = 1):
    #返回<num>个随机作业排列，包括当前的排列
    candidates = [perm]
    for i in range(num):
        candidate = perm[:]
        random.shuffle(candidate)
        candidates.append(candidate)
    return candidates

**第二领域考虑交换排列中的任何两个作业。**通过使用`itertools`包中的`combinations`函数，我们可以轻松地遍历每对索引，并创建对应于交换位于每个索引处的作业的新排列。从某种意义上说，这个邻域产生的排列与我们开始时的排列非常相似。

In [11]:
def neighbours_swap(data, perm):
    #返回与交换每一对作业相对应的排列
    candidates = [perm]
    for (i,j) in combinations(range(len(perm)), 2):
        candidate = perm[:]
        candidate[i], candidate[j] = candidate[j], candidate[i]
        candidates.append(candidate)
    return candidates

**第三领域使用特定于当前问题的信息。**我们发现空闲时间最多的作业，并考虑以各种可能的方式交换它们。我们采用的值`size`是我们考虑的作业数量：`size`最空闲的作业。

该函数的执行过程如下：
1. 计算排列中每个作业的空闲时间；
2. 计算出`size`空闲时间最多的作业列表；
3. 通过考虑已确定的最闲置作业的每个排列来构造邻域。为了找到排列，我们利用`itertools`包中的`permutations`函数。


In [12]:
def neighbours_idle(data, perm, size=4):
    #返回<size>最空闲作业的排列
    candidates = [perm]

    #计算每个作业的空闲时间
    sol = flow.compile_solution(data, perm)
    results = []

    for i in range(len(data)):
        finish_time = sol[-1][i] + data[perm[i]][-1]
        idle_time = (finish_time - sol[0][i]) - sum([t for t in data[perm[i]]])
        results.append((idle_time, i))
        
    #以<size>最空闲的作业为例
    subset = [job_ind for (idle, job_ind) in reversed(sorted(results))][:size]
    
    #枚举空闲作业的排列
    for ordering in permutations(subset):
        candidate = perm[:]
        for i in range(len(ordering)):
            candidate[subset[i]] = perm[ordering[i]]
        candidates.append(candidate)

    return candidates

**第四邻域通常称为大邻域搜索（LNS）。**直观上，LNS通过隔离考虑当前排列的较小子集来工作，找到工作子集的最佳排列将为我们提供LNS领域的单个候选者。

通过对特定大小的几个（或全部）子集重复此过程，我们可以增加邻域中的候选者数量。我们通过`MAX_LNS_NEIGHBOURHOODS`参数考虑限制的数目，因为邻居的数目可以快速增长。

LNS的计算过程如下：
1. 计算随机的作业集列表，我们将考虑使用`itertools`包的`combinations`功能进行交换；
2. 遍历子集以找到每个作业的最佳排列。我们在上面已经看到了类似的代码，可以循环访问最空闲的作业的所有排列。此处的主要区别在于，**我们仅记录该子集的最佳排列**，因为通过为所考虑的工作的每个子集选择一个排列来构建更大的邻域。

如果我们将`size`参数设置为作业数，则将考虑每个排列并选择最佳排列。但是，实际上，我们需要将子集的大小限制为3或4；任何较大的值都会导致该`neighbours_LNS`功能花费大量时间。

In [13]:
def neighbours_LNS(data, perm, size = 2):
    #返回大LNS搜索领域
    candidates = [perm]

    #限制社区的数量，以防有太多的工作机会
    neighbourhoods = list(combinations(range(len(perm)), size))
    random.shuffle(neighbourhoods)
    
    for subset in neighbourhoods[:flow.MAX_LNS_NEIGHBOURHOODS]:

        #记录每个领域的最佳排列
        best_make = flow.makespan(data, perm)
        best_perm = perm

        #枚举所选邻域的每个排列
        for ordering in permutations(subset):
            candidate = perm[:]
            for i in range(len(ordering)):
                candidate[subset[i]] = perm[ordering[i]]
            res = flow.makespan(data, candidate)
            if res < best_make:
                best_make = res
                best_perm = candidate

        #将最佳候选人记录为更大社区的一部分
        candidates.append(best_perm)

    return candidates

### 3.2 Heuristics

启发式方法从一组提供的候选中返回单个候选排列。试探法还被授予对问题数据的访问权限，以便评估哪个候选者可能是首选。

我们考虑的**第一个启发式方法是`heur_random`**。这种试探法从列表中随机选择一个候选人，而不评估哪个人可能更受欢迎：

In [14]:
def heur_random(data, candidates):
    #返回一个随机的候选选项
    return random.choice(candidates)

**下一个启发式`heur_hillclimbing`使用另一个极端**。与其随机选择一个候选者，不如选择具有最广有效期的候选者。

请注意，列表`scores`将包含以下形式的元组，`(make,perm)`其中`make`为置换的`perm`的`makepan`值。对这样的列表进行排序会将具有最佳生成时间的元组放在列表的开头；从这个元组中，我们返回排列。

In [15]:
def heur_hillclimbing(data, candidates):
    #返回列表中的最佳候选者
    scores = [(flow.makespan(data, perm), perm) for perm in candidates]
    return sorted(scores)[0][1]

**最终的启发式`heur_random_hillclimbing`结合了上面两种方法**。

执行本地搜索时，您可能并不总是希望选择一个随机的候选者，甚至最好的候选者。通过选择概率为0.5的最佳候选者，然后选择概率为0.25的次优候选者，heur_random_hillclimbing启发式方法返回“相当好”的解决方案，依此类推。

while循环实质上是在每次迭代时翻转硬币，以查看它是否应继续增加索引（限制列表的大小）。选择的最终索引对应于启发式选择的候选。

In [16]:
def heur_random_hillclimbing(data, candidates):
    #返回与排序的质量成正比的概率的候选人
    scores = [(flow.makespan(data, perm), perm) for perm in candidates]
    i = 0
    while (random.random() < 0.5) and (i < len(scores) - 1):
        i += 1
    return sorted(scores)[i][1]

由于makepan是我们正在尝试优化的标准，因此爬坡将引导本地搜索过程转向具有更佳makepan的解决方案。 引入随机性使我们能够探索邻居，而不是在每一步都盲目追求最佳外观的解决方案。

### 3.3 动态策略选择

本地搜索的核心是**使用特定的启发式和邻域函数从一种解决方案跳到另一种解决方案**。 

我们如何选择一组选项而不是另一组？ 

实际上，在搜索过程中切换策略经常会有所回报。我们使用的动态策略选择将在试探性功能和邻域功能的组合之间切换，以尝试动态地转移到效果最佳的那些策略上。对我们来说，策略是启发式和邻域函数（包括其参数值）的特定配置。

1. 首先，我们的代码构建了我们在求解过程中要考虑的策略范围。在策略初始化中，我们使用`functools`包中的`partial`函数为每个街区部分分配参数。
2. 此外，我们构造了启发式函数列表。
3. 最后，我们使用乘积运算符将邻域和启发式函数的每种组合添加为新策略。


In [19]:
################
## Strategies ##
#################################################
## 策略是Neighbourhoods生成器（用于计算下一组候选对象）
## 和Heuristics计算（用于选择最佳候选对象）的特定配置。
##

STRATEGIES = []

#使用命名字典比使用字典更干净。 例如，strategy ['name']与strategy.name
Strategy = namedtuple('Strategy', ['name', 'neighbourhood', 'heuristic'])

def initialize_strategies():

    global STRATEGIES

    #定义我们要使用的邻域（和参数）
    neighbourhoods = [
        ('Random Permutation', partial(neighbours_random, num=100)),
        ('Swapped Pairs', neighbours_swap),
        ('Large Neighbourhood Search (2)', partial(neighbours_LNS, size=2)),
        ('Large Neighbourhood Search (3)', partial(neighbours_LNS, size=3)),
        ('Idle Neighbourhood (3)', partial(neighbours_idle, size=3)),
        ('Idle Neighbourhood (4)', partial(neighbours_idle, size=4)),
        ('Idle Neighbourhood (5)', partial(neighbours_idle, size=5))
    ]

    #定义我们要使用的启发式
    heuristics = [
        ('Hill Climbing',heur_hillclimbing),
        ('Random Selection',heur_random),
        ('Biased Random Selection',heur_random_hillclimbing)
    ]

    #组合每一个邻域和启发式策略
    for (n, h) in product(neighbourhoods, heuristics):
        STRATEGIES.append(Strategy("%s / %s" % (n[0], h[0]), n[1], h[1]))

一旦定义了策略，我们就不必在搜索过程中坚持使用单个选项。取而代之的是，我们随机选择任何一种策略，但要根据策略的执行情况对选择进行加权。

我们在下面描述权重，但是对于`pick_strategy`功能，我们只需要策略列表和相应的相对权重列表（任何数字都可以）。

In [18]:
def pick_strategy(strategies, weights):
    #根据权重选择随机策略：轮盘赌选择并非完全随机选择策略，
    #而是将随机选择偏向于过去效果良好的策略（根据权重值）。
    total = sum([weights[strategy] for strategy in strategies])
    pick = random.uniform(0, total)
    count = weights[strategies[0]]

    i = 0
    while pick > count:
        count += weights[strategies[i+1]]
        i += 1

    return strategies[i]

为了选择具有给定权重的随机策略，我们统一选择一个介于0和所有权重之和之间的数字。随后，我们找到最低的索引i这样索引的所有权重之和小于i大于我们选择的随机数。该技术有时称为**轮盘赌选择**，将为我们随机选择一项策略，并为那些权重较高的策略提供更大的机会。

剩下的就是描述在寻找解决方案的过程中如何增加权重。这在**`solve`函数**的主`while`循环中以规则的时间间隔发生（由`TIME_INCREMENT`变量定义）。
```python
        if time.time() > time_last_switch + TIME_INCREMENT:

            time_last_switch = time.time()
```
回想一下，`strat_improvements`存储策略已进行的所有改进的总和，而`strat_time_spent存`储最后一次间隔中给出策略的时间。 我们将每种策略所花费的总时间进行归一化，以衡量每个策略在上一个时间间隔内的效果。由于策略可能根本没有机会执行，因此我们选择少量时间作为默认值。

```python
            results = sorted([
                (float(strat_improvements[s]) / max(0.001, strat_time_spent[s]), s)
                for s in STRATEGIES])
```

现在我们已经对每个策略的执行情况进行了排名，我们将$k$添加到最佳策略的权重（假设我们有$k$个策略），将$k-1$添加到次佳策略，等等。每个策略都有其自己的权重增加，列表中最差的策略只会增加1。
```python
            # Boost the weight for the successful strategies
            for i in range(len(STRATEGIES)):
                strat_weights[results[i][1]] += len(STRATEGIES) - i
```

作为一项额外措施，我们人为地提高了所有未使用的策略。 这样做是为了确保我们不会完全忘记策略。 尽管一种策略在开始时似乎表现不佳，但后来在搜索中却证明是非常有用的。

```python
                if results[i][0] == 0:
                    strat_weights[results[i][1]] += len(STRATEGIES)
```

最后，我们输出有关策略排名的一些信息（如果设置了`DEBUG_SWITCH`标志），然后在下一个时间间隔重置`strat_improvements`和`strat_time_spent`变量。

```python
            if DEBUG_SWITCH:
                print ("\nComputing another switch...")
                print ("Best: %s (%d)" % (results[0][1].name, results[0][0]))
                print ("Worst: %s (%d)" % (results[-1][1].name, results[-1][0]))
                print (results)
                print (sorted([strat_weights[STRATEGIES[i]] 
                              for i in range(len(STRATEGIES))]))

            strat_improvements = {strategy: 0 for strategy in STRATEGIES}
            strat_time_spent = {strategy: 0 for strategy in STRATEGIES}
```

接下来，我们以小文件来测试一下调度算法的运行结果。

In [21]:
!python flow.py tai20_5.txt


Parsing...

Solving...
 10 %
 20 %
 30 %
 40 %
 50 %
 60 %
 70 %
 80 %
 90 %
 100 %


Went through 3625 iterations.

(usage) Strategy:
(264) 	Idle Neighbourhood (3) / Hill Climbing
(250) 	Large Neighbourhood Search (2) / Random Selection
(223) 	Idle Neighbourhood (4) / Hill Climbing
(232) 	Random Permutation / Hill Climbing
(213) 	Idle Neighbourhood (5) / Hill Climbing
(247) 	Idle Neighbourhood (4) / Biased Random Selection
(216) 	Idle Neighbourhood (5) / Biased Random Selection
(230) 	Swapped Pairs / Hill Climbing
(234) 	Swapped Pairs / Biased Random Selection
(217) 	Idle Neighbourhood (3) / Biased Random Selection
(154) 	Large Neighbourhood Search (3) / Random Selection
(160) 	Large Neighbourhood Search (2) / Biased Random Selection
(188) 	Large Neighbourhood Search (2) / Hill Climbing
(146) 	Large Neighbourhood Search (3) / Biased Random Selection
(164) 	Large Neighbourhood Search (3) / Hill Climbing
(129) 	Idle Neighbourhood (3) / Random Selection
(93) 	Random Permutation / Biased

## 总结

在本练习中，我们看到了用相对少量的代码可以解决流水车间调度的复杂优化问题的方法。很难找到诸如流水车间之类的大型优化问题的最佳解决方案。在这种情况下，我们可以求助于近似技术（例如局部搜索）来计算足够好的解。通过局部搜索，我们可以从一种解决方案转到另一种解决方案，以期找到质量好的解决方案。

局部搜索的一般直觉可以应用于各种各样的问题。 我们专注于：
- 从一个候选解决方案生成问题的相关解决方案的邻域
- 建立评估和比较解决方案的方法。 

有了这两个组成部分，当难以找到最优方案时，我们可以使用局部搜索范式找到有价值的解决方案。

我们没有使用任何一种策略来解决问题，而是看到了如何在解决过程中动态选择一种策略来进行转移。这种简单而强大的技术使程序能够混合和匹配针对当前问题的部分策略，这也意味着开发人员不必手工定制该策略。